In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import requests
from PIL import Image
from io import BytesIO
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [28]:
dataFrame = pd.read_csv("transfermarkt/appearances.csv")

In [32]:
dataFrame2 = pd.read_csv("transfermarkt/players.csv")

In [36]:
dataFrame3 = pd.read_csv("transfermarkt/clubs.csv")

In [38]:
# dataFrame3'teki club_id sütununun adını player_club_id olarak değiştirme
dataFrame3 = dataFrame3.rename(columns={'club_id': 'player_club_id'})
dataFrame3 = dataFrame3.rename(columns={'name': 'club_name'})

In [46]:
# Önce dataFrame ve dataFrame2'yi birleştir
merged_data = pd.merge(dataFrame, dataFrame2, on='player_id', how='inner')

# Şimdi merged_data ile dataFrame3'ü birleştir
oyuncuData = pd.merge(merged_data, dataFrame3, on='player_club_id', how='inner')

In [47]:
oyuncuData['date'] = pd.to_datetime(oyuncuData['date'], errors='coerce')

# 11 Ağustos 2023 tarihinden sonraki verileri filtreleme
oyuncuData = oyuncuData[oyuncuData['date'] > '2013-08-14']

In [50]:
oyuncuData = oyuncuData[oyuncuData['competition_id'] == 'TR1']

In [52]:
import pandas as pd

# Sezon tarih aralıkları
seasons = {
    '13-14': ('2013-08-15', '2014-05-19'),
    '14-15': ('2014-08-28', '2015-05-31'),
    '15-16': ('2015-08-13', '2016-05-20'),
    '16-17': ('2016-08-18', '2017-06-04'),
    '17-18': ('2017-08-10', '2018-05-20'),
    '18-19': ('2018-08-16', '2019-05-27'),
    '19-20': ('2019-08-15', '2020-07-27'),
    '20-21': ('2020-09-10', '2021-05-15'),
    '21-22': ('2021-08-12', '2022-05-23'),
    '22-23': ('2022-08-04', '2023-07-08'),
    '23-24': ('2023-08-10', '2024-05-27'),
    '24-25': ('2024-08-08', '2025-05-27') 
}

# Sezon başlangıç ve bitiş tarihlerini listeye çevirin
season_start_dates = [pd.to_datetime(dates[0]) for dates in seasons.values()]
season_end_dates = [pd.to_datetime(dates[1]) for dates in seasons.values()]
season_labels = list(seasons.keys())

# Verideki tarihleri datetime formatına çevirin
oyuncuData['date'] = pd.to_datetime(oyuncuData['date'])

# Tarihlere göre sezonları belirleyin
oyuncuData['season'] = pd.cut(oyuncuData['date'],
                              bins=season_start_dates + [season_end_dates[-1]],
                              labels=season_labels,
                              right=False)  # Başlangıç tarihini kapsasın

# Sonucu kontrol edin
print(oyuncuData[['player_name', 'date', 'season']].head())


            player_name       date season
136536    Volkan Okumak 2013-08-16  13-14
136537  Björn Vleminckx 2013-08-16  13-14
136538     Hakan Arikan 2013-08-16  13-14
136539     Emrah Başsan 2013-08-16  13-14
136540             Tita 2013-08-16  13-14


In [60]:
# Oyuncu ve sezon bazında verileri grupla ve istatistikleri toplula
oyuncuData = oyuncuData.groupby(['player_id', 'season'], as_index=False, observed=True).agg({
    'net_transfer_record': 'first',
    'image_url':'first',
    'player_club_id': 'first',
    'club_name': 'first',
    'position': 'first',
    'player_name': 'first',
    'yellow_cards': 'sum',
    'red_cards': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'minutes_played': 'sum',
    'squad_size': 'first',
    'average_age': 'first',
    'foreigners_number': 'first',
    'foreigners_percentage': 'first',
    'national_team_players': 'first',
    'country_of_citizenship': 'first',
    'foot': 'first',
    'height_in_cm': 'first',
    'date': 'first'
}).reset_index(drop=True)

In [66]:
# Dakika başına gol
oyuncuData['goals_per_minute'] = oyuncuData['goals'] / oyuncuData['minutes_played']

# Dakika başına asist
oyuncuData['assists_per_minute'] = oyuncuData['assists'] / oyuncuData['minutes_played']

# Dakika başına gol ve asist toplamı
oyuncuData['goals_assists_per_minute'] = (oyuncuData['goals'] + oyuncuData['assists']) / oyuncuData['minutes_played']

# Dakika başına sarı kart
oyuncuData['yellow_cards_per_minute'] = oyuncuData['yellow_cards'] / oyuncuData['minutes_played']

# Dakika başına kırmızı kart
oyuncuData['red_cards_per_minute'] = oyuncuData['red_cards'] / oyuncuData['minutes_played']

# Dakika başına toplam kart (sarı + kırmızı)
oyuncuData['cards_per_minute'] = (oyuncuData['yellow_cards'] + oyuncuData['red_cards']) / oyuncuData['minutes_played']

In [70]:
oyuncuData['country_of_citizenship'] = oyuncuData['country_of_citizenship'].replace('Turkey', 'Türkiye')

In [72]:
oyuncuData.describe()

,player_id,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,total_market_value,squad_size,average_age,...,foreigners_percentage,national_team_players,height_in_cm,date,goals_per_minute,assists_per_minute,goals_assists_per_minute,yellow_cards_per_minute,red_cards_per_minute,cards_per_minute
count,6.349000e+03,6349.000000,6349.000000,6349.00000,6349.000000,6349.000000,6349.000000,0.0,6349.000000,6012.000000,...,5654.000000,6349.000000,6218.000000,6349,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000,6349.000000
mean,2.104081e+05,4384.489841,2.641518,0.07009,1.564498,1.234683,1129.919200,NaN,27.004883,25.377745,...,42.800778,5.235628,181.814731,2019-05-05 05:43:36.790045696,0.001322,0.001081,0.002402,0.002916,0.000090,0.003006
min,1.090000e+02,6.000000,0.000000,0.00000,0.000000,0.000000,1.000000,NaN,0.000000,21.000000,...,2.400000,0.000000,160.000000,2013-08-16 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.649000e+04,449.000000,0.000000,0.00000,0.000000,0.000000,276.000000,NaN,27.000000,25.100000,...,43.800000,1.000000,177.000000,2016-08-20 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.468440e+05,1506.000000,2.000000,0.00000,0.000000,0.000000,967.000000,NaN,29.000000,25.800000,...,46.900000,5.000000,182.000000,2019-08-18 00:00:00,0.000000,0.000000,0.000982,0.001847,0.000000,0.001919
75%,2.917500e+05,6646.000000,4.000000,0.00000,2.000000,2.000000,1879.000000,NaN,31.000000,26.300000,...,51.600000,7.000000,187.000000,2022-08-05 00:00:00,0.001667,0.001499,0.003431,0.003295,0.000000,0.003398
max,1.264192e+06,44006.000000,17.000000,2.00000,29.000000,19.000000,3420.000000,NaN,41.000000,28.400000,...,57.100000,15.000000,205.000000,2024-09-19 00:00:00,0.142857,0.100000,0.142857,1.000000,0.071429,1.000000
std,2.090031e+05,6921.344309,2.747288,0.26972,2.963197,2.000564,919.467812,NaN,8.266654,1.588158,...,13.200074,4.296390,6.459019,NaN,0.003741,0.003016,0.004928,0.019336,0.001374,0.019380


In [74]:
oyuncuData.isnull().sum()

player_id                      0
season                         0
net_transfer_record            0
image_url                      0
player_club_id                 0
club_name                      0
position                       0
player_name                    0
yellow_cards                   0
red_cards                      0
goals                          0
assists                        0
minutes_played                 0
total_market_value          6349
squad_size                     0
average_age                  337
foreigners_number              0
foreigners_percentage        695
national_team_players          0
country_of_citizenship       128
foot                          88
height_in_cm                 131
date                           0
goals_per_minute               0
assists_per_minute             0
goals_assists_per_minute       0
yellow_cards_per_minute        0
red_cards_per_minute           0
cards_per_minute               0
dtype: int64

In [76]:
# Sayısal verileri medyan ile doldurma
for column in oyuncuData.select_dtypes(include=['float64', 'int64']).columns:
    oyuncuData[column].fillna(oyuncuData[column].median(), inplace=True)

# Kategorik verileri "Bilinmiyor" ile doldurma
for column in oyuncuData.select_dtypes(include=['object']).columns:
    oyuncuData[column].fillna('Bilinmiyor', inplace=True)


C:\TEMP\ipykernel_16644\4005806511.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oyuncuData[column].fillna(oyuncuData[column].median(), inplace=True)
C:\TEMP\ipykernel_16644\4005806511.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [78]:
oyuncuData['club_name'].unique()

array(['Eskisehirspor', 'Konyaspor', 'Genclerbirligi Ankara',
       'Antalyaspor', 'Beşiktaş Jimnastik Kulübü',
       'Galatasaray Spor Kulübü', 'Akhisarspor', 'Ankaraspor',
       'Kasımpaşa Spor Kulübü', 'Gaziantepspor (- 2020)',
       'Çaykur Rizespor Kulübü', 'Fenerbahçe Spor Kulübü',
       'Kardemir Karabükspor', 'Elazigspor', 'Kayseri Erciyesspor',
       'Trabzonspor Kulübü', 'Sivasspor Kulübü',
       'İstanbul Başakşehir Futbol Kulübü', 'Bursaspor',
       'Mersin Talimyurdu SK', 'Göztepe Sportif Yatırımlar A.Ş.',
       'Yeni Malatyaspor', 'MKE Ankaragücü', 'Erzurumspor FK',
       'Balikesirspor', 'Kayserispor Kulübü', 'Alanyaspor',
       'Ümraniyespor', 'Adana Demirspor Kulübü', 'Fatih Karagümrük',
       'Denizlispor', 'Gaziantep Futbol Kulübü A.Ş.', 'Istanbulspor',
       'Altay SK', 'Bodrumspor Spor Faaliyetleri Anonim Şirketi',
       'Eyüp Spor Kulübü', 'Hatayspor Futbol Kulübü', 'Pendikspor',
       'Adanaspor', 'Giresunspor', 'Samsunspor'], dtype=object)

In [199]:
# Takım ve sezon bilgilerini burada belirleyin
takim = 'Samsunspor'  # Örneğin: 'Fenerbahçe'
sezonlar = ['13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24','24-25']  # Örneğin: ['2013/2014', '2014/2015', '2015/2016']
goal_color = 'red'   # Goller için renk
assist_color = 'white'  # Asistler için renk

# Klasörleri oluşturma
root_folder = 'Takımlar'
os.makedirs(root_folder, exist_ok=True)

# Her sezon için döngü
for sezon in sezonlar:
    # Belirli bir takım ve sezon için veriyi filtrele
    takim_sezon = oyuncuData[(oyuncuData['season'] == sezon) & (oyuncuData['club_name'] == takim)]

    # Takım klasörünü oluştur
    sezon_folder = os.path.join(root_folder, sezon)
    takim_folder = os.path.join(sezon_folder, takim)
    os.makedirs(takim_folder, exist_ok=True)

    if not takim_sezon.empty:
        # Golleri, asistleri, sarı kart ve kırmızı kartları toplama
        goller_asistler = takim_sezon.groupby('player_name').agg({
            'goals': 'sum',
            'assists': 'sum',
            'yellow_cards': 'sum',
            'red_cards': 'sum',
            'minutes_played': 'sum',
            'image_url': 'first',
            'goals_per_minute': 'first',
            'assists_per_minute': 'first',
            'yellow_cards_per_minute': 'first',
            'red_cards_per_minute': 'first',
            'cards_per_minute': 'first',
            'position': 'first',
            'foot': 'first',
            'height_in_cm': 'first',
            'country_of_citizenship': 'first',
            'national_team_players' : 'first'
        }).reset_index()
        pozisyon_sayilari = takim_sezon['position'].value_counts()
        # 1. Oynama Süreleri (Dakika) Grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        plt.bar(goller_asistler['player_name'], goller_asistler['minutes_played'], color='purple')
        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Oynama Süreleri (Dakika)')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Toplam Dakika')
        plt.xticks(rotation=90)
        plt.tight_layout()

        # Oyuncu resimlerini ekleme
        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)
            imagebox = OffsetImage(img, zoom=0.5)
            ab = AnnotationBbox(imagebox, (i, goller_asistler['minutes_played'].iloc[i] + 50), frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, goller_asistler['minutes_played'].max() + 400)
        oynama_sureleri_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_oynama_sureleri.png')
        plt.savefig(oynama_sureleri_dosyasi, dpi=600)
        plt.close()

        # 2. Goller ve Asist Grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        bar_width = 0.35
        x = np.arange(len(goller_asistler['player_name']))

        plt.bar(x - bar_width / 2, goller_asistler['goals'], width=bar_width, label='Goller', color=goal_color)
        plt.bar(x + bar_width / 2, goller_asistler['assists'], width=bar_width, label='Asistler', color=assist_color)

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Golleri ve Asistleri')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Toplam Gol ve Asist Sayısı')
        plt.xticks(x, goller_asistler['player_name'], rotation=90)
        plt.legend()
        plt.tight_layout()

        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)
            imagebox = OffsetImage(img, zoom=0.5)
            ab = AnnotationBbox(imagebox, (i, max(goller_asistler['goals'].iloc[i], goller_asistler['assists'].iloc[i]) + 0.1), frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, max(goller_asistler['goals'].max(), goller_asistler['assists'].max()) + 2)
        goller_asistler_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_goller_asistler.png')
        plt.savefig(goller_asistler_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu için grafikler başarıyla kaydedildi.')

        # 3. Dakika Başına Gol ve Asist Grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        plt.bar(x - bar_width/2, goller_asistler['goals_per_minute'], width=bar_width, label='Dakika Başına Gol', color=goal_color)
        plt.bar(x + bar_width/2, goller_asistler['assists_per_minute'], width=bar_width, label='Dakika Başına Asist', color=assist_color)

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Dakika Başına Gol ve Asist Sayıları')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Dakika Başına Gol ve Asist')
        plt.xticks(x, goller_asistler['player_name'], rotation=90)
        plt.legend()
        plt.tight_layout()

        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)
            imagebox = OffsetImage(img, zoom=0.5)
            ab = AnnotationBbox(imagebox, (i, max(goller_asistler['goals_per_minute'].iloc[i], goller_asistler['assists_per_minute'].iloc[i]) + 0.00015), frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, max(goller_asistler['goals_per_minute'].max(), goller_asistler['assists_per_minute'].max()) + 0.005)
        dakika_basina_goller_asistler_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_dakika_basina_goller_asistler.png')
        plt.savefig(dakika_basina_goller_asistler_dosyasi, dpi=600)
        plt.close()

        # 4. Sarı ve Kırmızı Kart Grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        plt.bar(x - bar_width/2, goller_asistler['yellow_cards'], width=bar_width, label='Sarı Kartlar', color='yellow')
        plt.bar(x + bar_width/2, goller_asistler['red_cards'], width=bar_width, label='Kırmızı Kartlar', color='red')

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Sarı ve Kırmızı Kart Sayıları')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Toplam Kart Sayısı')
        plt.xticks(x, goller_asistler['player_name'], rotation=90)
        plt.legend()
        plt.tight_layout()

        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)
            imagebox = OffsetImage(img, zoom=0.5)
            ab = AnnotationBbox(imagebox, (i, max(goller_asistler['yellow_cards'].iloc[i], goller_asistler['red_cards'].iloc[i]) + 0.15), frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, max(goller_asistler['yellow_cards'].max(), goller_asistler['red_cards'].max()) + 1)
        kartlar_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_sari_kirmizi_kartlar.png')
        plt.savefig(kartlar_dosyasi, dpi=600)
        plt.close()


        # Dakika başına sarı ve kırmızı kartlar için grafik
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        plt.bar(x - bar_width/2, goller_asistler['yellow_cards_per_minute'], width=bar_width, label='Dakika Başına Sarı Kart', color='yellow')
        plt.bar(x + bar_width/2, goller_asistler['red_cards_per_minute'], width=bar_width, label='Dakika Başına Kırmızı Kart', color='red')

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Dakika Başına Sarı ve Kırmızı Kart Sayıları')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Dakika Başına Kart Sayısı')
        plt.xticks(x, goller_asistler['player_name'], rotation=90)
        plt.legend()
        plt.tight_layout()

        # Oyuncu resimlerini ekleme
        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)  # Resmi 50x50 boyutuna küçült
            imagebox = OffsetImage(img, zoom=0.5)  # Zoom ile görüntü boyutunu ayarlama
            ab = AnnotationBbox(imagebox, (i, max(goller_asistler['yellow_cards_per_minute'].iloc[i], goller_asistler['red_cards_per_minute'].iloc[i]) + 0.0002),  # Resmi bar üstüne yerleştir
                            frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, max(goller_asistler['yellow_cards_per_minute'].max(), goller_asistler['red_cards_per_minute'].max()) + 0.005)  # Biraz boşluk bırak
        dakika_basina_kartlar_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}dakika_basina_sari_kirmizi_kartlar.png')
        plt.savefig(dakika_basina_kartlar_dosyasi, dpi=600)
        plt.close()


        # Grafik oluşturma
        plt.figure(figsize=(10, 6))
        plt.gca().set_facecolor('#DCDCDC')
        sbn.barplot(x=pozisyon_sayilari.index, y=pozisyon_sayilari.values, palette='viridis')
        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Pozisyon Dağılımı')
        plt.xlabel('Pozisyon')
        plt.ylabel('Oyuncu Sayısı')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Pozisyon grafiğini kaydetme
        pozisyon_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_pozisyonlar.png')
        plt.savefig(pozisyon_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu pozisyon grafiği başarıyla kaydedildi.')


        # Ayak Kullanım Grafiği
        plt.figure(figsize=(10, 6))
        plt.gca().set_facecolor('#DCDCDC')
        sbn.countplot(x='foot', data=takim_sezon, palette='Set2')
        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Ayak Kullanım Dağılımı')
        plt.xlabel('Kullanılan Ayak')
        plt.ylabel('Oyuncu Sayısı')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Ayak kullanım grafiğini kaydetme
        foot_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_ayak_kullanimi.png')
        plt.savefig(foot_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu ayak kullanım grafiği başarıyla kaydedildi.')


        # Boy Grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')
        bar_width = 0.35
        x = np.arange(len(goller_asistler['player_name']))

        # Boyları bar grafiği olarak göster
        plt.bar(x, goller_asistler['height_in_cm'], width=bar_width, color='skyblue', label='Boy (cm)')

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Boy Dağılımı')
        plt.xlabel('Oyuncu Adı')
        plt.ylabel('Boy (cm)')
        plt.xticks(x, goller_asistler['player_name'], rotation=90)
        plt.legend()
        plt.tight_layout()

        # Oyuncu resimlerini ekleme
        for i, player in enumerate(goller_asistler['player_name']):
            photo_url = goller_asistler.loc[goller_asistler['player_name'] == player, 'image_url'].values[0]
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)  # Resmi 50x50 boyutuna küçült
            imagebox = OffsetImage(img, zoom=0.5)  # Zoom ile görüntü boyutunu ayarlama
            ab = AnnotationBbox(imagebox, (i, goller_asistler['height_in_cm'].iloc[i] + 5),  # Resmi bar üstüne yerleştir
                            frameon=False, box_alignment=(0.5, 0.1))
            plt.gca().add_artist(ab)

        plt.ylim(0, goller_asistler['height_in_cm'].max() + 50)  # Biraz boşluk bırak

        # Boy grafiğini kaydetme
        height_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_boy.png')
        plt.savefig(height_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu boy grafiği başarıyla kaydedildi.')



        # Pozisyona göre ayak kullanım grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')

        # Pozisyon ve ayak kullanım verisini toplama
        ayak_kullanimi = goller_asistler.groupby(['position', 'foot']).size().unstack(fill_value=0)

        # Bar grafiği oluşturma
        ayak_kullanimi.plot(kind='bar', stacked=True, color=['lightblue', 'lightcoral'], ax=plt.gca())

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Pozisyona Göre Ayak Kullanımı')
        plt.xlabel('Pozisyon')
        plt.ylabel('Oyuncu Sayısı')
        plt.xticks(rotation=45)
        plt.legend(title='Ayak', labels=['Sağ', 'Sol'])
        plt.tight_layout()

        # Grafiği kaydetme
        ayak_kullanimi_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}pozisyon_ayak_kullanimi.png')
        plt.savefig(ayak_kullanimi_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu pozisyona göre ayak kullanımı grafiği başarıyla kaydedildi.')



        # Pozisyona göre boy grafiği
        plt.figure(figsize=(12, 6))
        plt.gca().set_facecolor('#DCDCDC')

        # Pozisyona göre boy verisini toplama
        ortalama_boy = goller_asistler.groupby('position')['height_in_cm'].mean()

        # Bar grafiği oluşturma
        ortalama_boy.plot(kind='bar', color='skyblue', ax=plt.gca())

        plt.title(f'{takim} Oyuncularının {sezon} Sezonundaki Pozisyona Göre Ortalama Boyları')
        plt.xlabel('Pozisyon')
        plt.ylabel('Ortalama Boy (cm)')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Grafiği kaydetme
        boy_grafik_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_pozisyona_gore_boy.png')
        plt.savefig(boy_grafik_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu pozisyona göre boy grafiği başarıyla kaydedildi.')


    

       # Seçilen takımın oyuncularını filtrele
        takim_oyunculari = oyuncuData[(oyuncuData['club_name'] == takim) & (oyuncuData['season'] == sezon)]

        # Ülke sayımını yapma
        ulke_sayilari = takim_oyunculari['country_of_citizenship'].value_counts()

        # Sadece mevcut ülkeleri filtreleme (zaten 0 olanları value_counts çıkarmaz)
        ulke_sayilari = ulke_sayilari[ulke_sayilari > 0]

        # Grafik oluşturma
        plt.figure(figsize=(10, 6))
        plt.gca().set_facecolor('#DCDCDC')  # Gri renk
        sbn.barplot(x=ulke_sayilari.index, y=ulke_sayilari.values, palette='magma')
        plt.title(f'{takim} Takımının {sezon} Sezonundaki Oyuncularının Ülke Dağılımı')
        plt.xlabel('Ülke')
        plt.ylabel('Oyuncu Sayısı')
        plt.xticks(rotation=90)
        plt.yticks(range(0, int(ulke_sayilari.max()) + 1))
        plt.tight_layout()

        # Grafiği kaydetme
        ulke_grafik_dosyasi = os.path.join(takim_folder, f'{takim}_{sezon}_ulke_dagilimi.png')
        plt.savefig(ulke_grafik_dosyasi, dpi=600)
        plt.close()

        print(f'{takim} {sezon} sezonu ülke grafiği başarıyla kaydedildi.')


Samsunspor 23-24 sezonu için grafikler başarıyla kaydedildi.


C:\TEMP\ipykernel_19036\4138417980.py:187: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sbn.barplot(x=pozisyon_sayilari.index, y=pozisyon_sayilari.values, palette='viridis')


Samsunspor 23-24 sezonu pozisyon grafiği başarıyla kaydedildi.


C:\TEMP\ipykernel_19036\4138417980.py:205: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sbn.countplot(x='foot', data=takim_sezon, palette='Set2')


Samsunspor 23-24 sezonu ayak kullanım grafiği başarıyla kaydedildi.
Samsunspor 23-24 sezonu boy grafiği başarıyla kaydedildi.
Samsunspor 23-24 sezonu pozisyona göre ayak kullanımı grafiği başarıyla kaydedildi.
Samsunspor 23-24 sezonu pozisyona göre boy grafiği başarıyla kaydedildi.


...

Samsunspor 24-25 sezonu pozisyon grafiği başarıyla kaydedildi.


C:\TEMP\ipykernel_19036\4138417980.py:205: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sbn.countplot(x='foot', data=takim_sezon, palette='Set2')


Samsunspor 24-25 sezonu ayak kullanım grafiği başarıyla kaydedildi.
Samsunspor 24-25 sezonu boy grafiği başarıyla kaydedildi.
Samsunspor 24-25 sezonu pozisyona göre ayak kullanımı grafiği başarıyla kaydedildi.
Samsunspor 24-25 sezonu pozisyona göre boy grafiği başarıyla kaydedildi.


C:\TEMP\ipykernel_19036\4138417980.py:322: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sbn.barplot(x=ulke_sayilari.index, y=ulke_sayilari.values, palette='magma')


Samsunspor 24-25 sezonu ülke grafiği başarıyla kaydedildi.


In [82]:
# Sezonlar listesi
last_10_seasons = ['13-14','14-15','15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24', '24-25']

# Sadece son 10 sezondaki verileri filtreleme
son_10_sezon_data = oyuncuData[oyuncuData['season'].isin(last_10_seasons)]

# Klasör yapısını kontrol et ve yoksa oluştur
def klasor_olustur(dizin):
    if not os.path.exists(dizin):
        os.makedirs(dizin)

# Her takım için genel katkı grafiğini kaydetme fonksiyonu (tüm sezonlar bir arada)
def genel_grafik_olustur_ve_kaydet(takim, data):
    # Her oyuncunun toplam gol ve asist katkısını hesapla (tüm sezonlar genelinde)
    katkilar = data.groupby('player_name').agg({'goals': 'sum', 'assists': 'sum', 'image_url': 'first'}).reset_index()

    # Oyunculara göre sıralama
    katkilar = katkilar.sort_values(by=['goals', 'assists'], ascending=False).head(10)

    # Grafik oluşturma
    plt.figure(figsize=(10, 6))
    
    # İki bar oluşturmak için indeksleri ayarlıyoruz
    bar_width = 0.4
    indices = np.arange(len(katkilar))

    # Gol barları
    plt.barh(indices - bar_width/2, katkilar['goals'], height=bar_width, label='Goller', color='blue')

    # Asist barları
    plt.barh(indices + bar_width/2, katkilar['assists'], height=bar_width, label='Asistler', color='green')

    plt.yticks(indices, katkilar['player_name'])
    plt.xlabel('Katkı Sayısı')
    plt.title(f'{takim} - Tüm Sezonlar En Çok Katkı Yapan 10 Oyuncu')
    plt.legend()

    # Oyuncu resimlerini ekleme (isteğe bağlı)
    for i, player in enumerate(katkilar['player_name']):
        photo_url = katkilar.loc[katkilar['player_name'] == player, 'image_url'].values[0]
        
        try:
            # Resmi indirme ve boyutlandırma
            response = requests.get(photo_url)
            img = Image.open(BytesIO(response.content))
            img = img.resize((50, 50), Image.LANCZOS)  # Resmi 50x50 boyutuna küçült

            # Görüntüyü grafikte yerleştirme
            imagebox = OffsetImage(img, zoom=0.5)
            ab = AnnotationBbox(imagebox, (-2, i),  # Bar üstüne yerleştir
                                frameon=False, box_alignment=(1.1, 0.5))  # Konum ve hizalama
            plt.gca().add_artist(ab)
        except Exception as e:
            print(f"Resim yüklenirken hata oluştu: {e}")

    # Grafik dosyasını Takımlar/genel/{takim} klasörüne kaydet
    dizin = f'Takımlar/genel/{takim}'
    klasor_olustur(dizin)
    plt.savefig(f'{dizin}/{takim}_genel_katkilar.png', bbox_inches='tight')
    plt.close()

# Her takım için genel katkıları hesaplayıp grafikleri kaydet
for takim in son_10_sezon_data['club_name'].unique():
    takim_data = son_10_sezon_data[son_10_sezon_data['club_name'] == takim]
    
    # Takım için genel katkı grafiğini oluştur ve kaydet
    genel_grafik_olustur_ve_kaydet(takim, takim_data)


In [90]:
# Sadece son 10 sezondaki verileri filtreleme (eğer bu blok ayrı çalışacaksa)
last_10_seasons = ['13-14','14-15','15-16', '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24', '24-25']
son_10_sezon_data = oyuncuData[oyuncuData['season'].isin(last_10_seasons)]

# Klasör yapısını kontrol et ve yoksa oluştur
def klasor_olustur(dizin):
    if not os.path.exists(dizin):
        os.makedirs(dizin)

# Dakika başına gol ve asist hesaplayan fonksiyon
def dakika_basina_grafik_olustur_ve_kaydet(takim, data):
    # Her oyuncunun dakika başına gol ve asist katkısını hesapla
    katkilar = data.groupby('player_name').agg({
        'goals': 'sum',
        'assists': 'sum',
        'minutes_played': 'sum',
        'image_url': 'first'
    }).reset_index()

    # Dakika başına katkılar
    katkilar['goals_per_min'] = katkilar['goals'] / katkilar['minutes_played']
    katkilar['assists_per_min'] = katkilar['assists'] / katkilar['minutes_played']

    # Oyunculara göre sıralama (dakika başına gol ve asistlere göre)
    katkilar = katkilar.sort_values(by=['goals_per_min', 'assists_per_min'], ascending=False).head(10)

    # Grafik oluşturma
    plt.figure(figsize=(10, 6))
    
    # İki bar oluşturmak için indeksleri ayarlıyoruz
    bar_width = 0.4
    indices = np.arange(len(katkilar))

    # Dakika başına gol barları
    plt.barh(indices - bar_width/2, katkilar['goals_per_min'], height=bar_width, label='Dakika Başına Goller', color='blue')

    # Dakika başına asist barları
    plt.barh(indices + bar_width/2, katkilar['assists_per_min'], height=bar_width, label='Dakika Başına Asistler', color='green')

    plt.yticks(indices, katkilar['player_name'])
    plt.xlabel('Dakika Başına Katkı')
    plt.title(f'{takim} - Tüm Sezonlar En Çok Dakika Başına Katkı Yapan 10 Oyuncu')
    plt.legend()

    # Grafik dosyasını Takımlar/genel/{takim} klasörüne kaydet
    dizin = f'Takımlar/genel/{takim}'
    klasor_olustur(dizin)
    plt.savefig(f'{dizin}/{takim}_dakika_basina_katkilar.png', bbox_inches='tight')
    plt.close()

# Her takım için dakika başına katkıları hesaplayıp grafikleri kaydet
for takim in son_10_sezon_data['club_name'].unique():
    takim_data = son_10_sezon_data[son_10_sezon_data['club_name'] == takim]
    
    # Takım için dakika başına katkı grafiğini oluştur ve kaydet
    dakika_basina_grafik_olustur_ve_kaydet(takim, takim_data)


In [98]:
# Klasör yapısını kontrol et ve yoksa oluştur
def klasor_olustur(dizin):
    if not os.path.exists(dizin):
        os.makedirs(dizin)

# Sarı ve kırmızı kartlar için grafik oluşturma (toplam kartlar)
def kart_grafik_olustur_ve_kaydet(takim, data):
    # Her oyuncunun toplam sarı ve kırmızı kartlarını hesapla
    katkilar = data.groupby('player_name').agg({
        'yellow_cards': 'sum',
        'red_cards': 'sum',
        'image_url': 'first'
    }).reset_index()

    # Oyunculara göre sıralama (kartlara göre)
    katkilar = katkilar.sort_values(by=['yellow_cards', 'red_cards'], ascending=False).head(10)

    # Grafik oluşturma
    plt.figure(figsize=(10, 6))
    
    # İki bar oluşturmak için indeksleri ayarlıyoruz
    bar_width = 0.4
    indices = np.arange(len(katkilar))

    # Sarı kart barları
    plt.barh(indices - bar_width/2, katkilar['yellow_cards'], height=bar_width, label='Sarı Kartlar', color='yellow')

    # Kırmızı kart barları
    plt.barh(indices + bar_width/2, katkilar['red_cards'], height=bar_width, label='Kırmızı Kartlar', color='red')

    plt.yticks(indices, katkilar['player_name'])
    plt.xlabel('Kart Sayısı')
    plt.title(f'{takim} - Tüm Sezonlar En Çok Kart Gören 10 Oyuncu')
    plt.legend()

    # Grafik dosyasını Takımlar/genel/{takim} klasörüne kaydet
    dizin = f'Takımlar/genel/{takim}'
    klasor_olustur(dizin)
    
    # Dosya adını açıkça belirt
    dosya_adi = f'{takim}_toplam_kartlar.png'
    plt.savefig(f'{dizin}/{dosya_adi}', bbox_inches='tight')
    print(f'{dosya_adi} dosyası kaydedildi.')
    plt.close()

# Dakika başına sarı ve kırmızı kart hesaplayan grafik oluşturma fonksiyonu
def dakika_basina_kart_grafik_olustur_ve_kaydet(takim, data):
    # Her oyuncunun dakika başına sarı ve kırmızı kart katkısını hesapla
    katkilar = data.groupby('player_name').agg({
        'yellow_cards': 'sum',
        'red_cards': 'sum',
        'minutes_played': 'sum',
        'image_url': 'first'
    }).reset_index()

    # Dakika başına katkılar
    katkilar['yellow_cards_per_min'] = katkilar['yellow_cards'] / katkilar['minutes_played']
    katkilar['red_cards_per_min'] = katkilar['red_cards'] / katkilar['minutes_played']

    # Oyunculara göre sıralama (dakika başına kartlara göre)
    katkilar = katkilar.sort_values(by=['yellow_cards_per_min', 'red_cards_per_min'], ascending=False).head(10)

    # Grafik oluşturma
    plt.figure(figsize=(10, 6))
    
    # İki bar oluşturmak için indeksleri ayarlıyoruz
    bar_width = 0.4
    indices = np.arange(len(katkilar))

    # Dakika başına sarı kart barları
    plt.barh(indices - bar_width/2, katkilar['yellow_cards_per_min'], height=bar_width, label='Dakika Başına Sarı Kartlar', color='yellow')

    # Dakika başına kırmızı kart barları
    plt.barh(indices + bar_width/2, katkilar['red_cards_per_min'], height=bar_width, label='Dakika Başına Kırmızı Kartlar', color='red')

    plt.yticks(indices, katkilar['player_name'])
    plt.xlabel('Dakika Başına Kart Sayısı')
    plt.title(f'{takim} - Tüm Sezonlar En Çok Dakika Başına Kart Gören 10 Oyuncu')
    plt.legend()

    # Grafik dosyasını Takımlar/genel/{takim} klasörüne kaydet
    dizin = f'Takımlar/genel/{takim}'
    klasor_olustur(dizin)
    
    # Dosya adını açıkça belirt
    dosya_adi = f'{takim}_dakika_basina_kartlar.png'
    plt.savefig(f'{dizin}/{dosya_adi}', bbox_inches='tight')
    print(f'{dosya_adi} dosyası kaydedildi.')
    plt.close()

# Her takım için kart verilerini hesaplayıp grafikleri kaydet
for takim in son_10_sezon_data['club_name'].unique():
    takim_data = son_10_sezon_data[son_10_sezon_data['club_name'] == takim]
    
    # Takım için toplam kart grafiğini oluştur ve kaydet
    kart_grafik_olustur_ve_kaydet(takim, takim_data)
    
    # Takım için dakika başına kart grafiğini oluştur ve kaydet
    dakika_basina_kart_grafik_olustur_ve_kaydet(takim, takim_data)


Eskisehirspor_toplam_kartlar.png dosyası kaydedildi.
Eskisehirspor_dakika_basina_kartlar.png dosyası kaydedildi.
Konyaspor_toplam_kartlar.png dosyası kaydedildi.
Konyaspor_dakika_basina_kartlar.png dosyası kaydedildi.
Genclerbirligi Ankara_toplam_kartlar.png dosyası kaydedildi.
Genclerbirligi Ankara_dakika_basina_kartlar.png dosyası kaydedildi.
Antalyaspor_toplam_kartlar.png dosyası kaydedildi.
Antalyaspor_dakika_basina_kartlar.png dosyası kaydedildi.
Beşiktaş Jimnastik Kulübü_toplam_kartlar.png dosyası kaydedildi.
Beşiktaş Jimnastik Kulübü_dakika_basina_kartlar.png dosyası kaydedildi.
Galatasaray Spor Kulübü_toplam_kartlar.png dosyası kaydedildi.
Galatasaray Spor Kulübü_dakika_basina_kartlar.png dosyası kaydedildi.
Akhisarspor_toplam_kartlar.png dosyası kaydedildi.
Akhisarspor_dakika_basina_kartlar.png dosyası kaydedildi.
Ankaraspor_toplam_kartlar.png dosyası kaydedildi.
Ankaraspor_dakika_basina_kartlar.png dosyası kaydedildi.
Kasımpaşa Spor Kulübü_toplam_kartlar.png dosyası kaydedild